In [1]:
import cv2
import numpy as np
import argparse
import csv
import os
import math
import random
import time
from copy import deepcopy
from operator import itemgetter
from itertools import *


In [2]:
def make_ellipse(image, row):
    center = (int(round(row[0])),int(round(row[1])))
    axes = (int(round(row[2])*2 ),int(round(row[3])*2) )
    angle = int(round(row[4]))
    color = (row[8],row[9],row[10])
    thickness = 4  #Do -1 if you want it filled in

    return cv2.ellipse(image, center, axes, angle, 0, 360, color, thickness) 

def make_ellipse2(image, ellipse):
    thickness = 4  #Do -1 if you want it filled in
    
    return cv2.ellipse(image, ellipse[1], ellipse[3], thickness) 

def make_ellipse3(image, ellipse,tubenum):
    thickness = 4  #Do -1 if you want it filled in
    center = (int(ellipse[0][1][0][0]), int(ellipse[0][1][0][1]) )
    
    cv2.putText(image,str(tubenum), center,cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), thickness)
    return cv2.ellipse(image, ellipse[0][1], ellipse[0][3], thickness) 


In [3]:
#This code is a recreation of the original tube code in Python. The input is a magno video and the output is a list of tubes

inmagnovid = 'virat7_JLJmagno.mp4'

vidcap = cv2.VideoCapture(inmagnovid) 
kernel = np.ones((10,10) , np.uint8)
magnovid = []
H=0
W=0
F=0

#Thresholds (at 50) the input magno video into a binary video 'magnovid'
while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        (dontcare, outframe) = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY)
        cv2.dilate(outframe,kernel, iterations =1)
        cv2.erode(outframe,kernel, iterations =1)
        H = frame.shape[0]
        W = frame.shape[1]
        F=F+1
        magnovid.append(outframe)

vidcap.release()
cv2.destroyAllWindows()


#Write the input video into a variable
invideo = 'virat7.mp4'
vidcap = cv2.VideoCapture(invideo) 
invid = []

while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        invid.append(frame)
        
vidcap.release()
cv2.destroyAllWindows()


#Writes the thresholded magno vid to a video file
vidwriter = cv2.VideoWriter('virat7_magnoOut.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%50==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnovid[i])
vidwriter.release()
cv2.destroyAllWindows()


#Each ellipse is put in the list as a tuple... (framenum, ellipse)
#Ellipse: (x,y) , (MAJORAXIS,minoraxis), angle
eList = []
for f in range(1,F): #for each frame
    M = cv2.cvtColor(magnovid[f], cv2.COLOR_BGR2GRAY);
    _, contours, hierarchy = cv2.findContours(M, 1, 2)
    
    for c in contours:
        if len(c) < 5:
            continue
        area = cv2.contourArea(c)
        if area < 100:
            continue
    
        ellipse = cv2.fitEllipse(c)
        eList.append( [f, ellipse, [0,0], [255,255,255] ] )
        

#Filtering the smallest ellipses (minsize for either dimension = 20pixels)
minsize = 20
eList = [e for e in eList if (e[1][1][0] > minsize) and (e[1][1][1]>minsize)]


#Putting into the familiar form. e = [x,y,a,b,phi,f,Vx,Vy,R,G,B] 
#Not actually using this yet bcuz it messes up when drawing on video
ellipseList=[]
for e in eList:
    ellipse = [e[1][0][0], e[1][0][1], e[1][1][1]/2, e[1][1][0]/2, e[1][2], e[0], 0,0,255,255,255]
    ellipseList.append(ellipse)
    

#Draw the ellipses on the video just to check...
magnotubes = deepcopy(invid)
for f in range(1,F):
    puthere = [e for e in eList if e[0]==f]
    
    for ellipse in puthere:
        make_ellipse2(magnotubes[f],ellipse)
        

#Writes the vid with tubes to a video file
vidwriter = cv2.VideoWriter('virat7_outstage0.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%100==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnotubes[i])
vidwriter.release()
cv2.destroyAllWindows()

FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [4]:


#Doing the first temporal associations. Should start to look like tubes
initialList = []
finalList = []
paths = [[]]
istart = 6
iend = istart+4
fstart = 11
fend = fstart+5
locthresh = 80
sizethresh = 50

while (fend < F):
    initialList = [e for e in eList if e[0]>=istart and e[0]<=iend]
    finalList =  [e for e in eList if e[0]>=fstart and e[0]<=fend]
    
    for ei in initialList:
        for ef in finalList:
            
            matchFlag = False
            r = math.sqrt((ef[1][0][0] - ei[1][0][0])**2 + (ef[1][0][1] - ei[1][0][1])**2 )
            sizediff = ef[1][1][1] - ei[1][1][1] #diff in major axis
            framediff = ef[0]-ei[0]
            
            if not(ei[2]==(0,0)):
                predX = ei[2][0]*framediff + ei[1][0][0]
                predY = ei[2][1]*framediff + ei[1][0][1]
                if (abs(predX-ef[1][0][0]) < 50) and (abs(predY-ef[1][0][1]) < 50):
                    matchFlag = True
            else:
                if (abs(sizediff)<sizethresh) and (r<locthresh):
                    matchFlag = True
            
            if (matchFlag):
                if ei[3]==[255,255,255]:
                    ei[3][0] = random.randint(0,255)
                    ei[3][1] = random.randint(0,255)
                    ei[3][2] = random.randint(0,255)
                ef[3][0] = ei[3][0]
                ef[3][1] = ei[3][1]
                ef[3][2] = ei[3][2]
                
                ef[2][0] = (ef[1][0][0]-ei[1][0][0])/framediff
                ef[2][1] = (ef[1][0][1]-ei[1][0][1])/framediff
                

                #This part searches for ei in existing paths and attaches ef if it finds it
                whichpath = 0
                for i in range(1,len(paths)): 
                    for ell in paths[i]:
                        if ei == ell:
                            whichpath = i
                            
                if not(whichpath==0):
                     paths[whichpath].append(ef)
                else:
                    newpath = []
                    newpath.append(ei)
                    newpath.append(ef)
                    paths.append(newpath)

    #print(fend)
    istart=istart+5
    iend=iend+5
    fstart=fstart+5
    fend=fend+5

def putTubesOnVid(tubevar, vidoutfile, withnums = False):
    
    if not withnums:
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                puthere.extend(P)

            for ellipse in puthere:
                make_ellipse2(outvid[f],ellipse)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    else:        
        #WITH NUMS    
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                T = [tubevar.index(path) for e in path if e[0]==f]
                puthere.append([P,T])

            for [ellipse,Tnum] in puthere:
                if ellipse:
                    make_ellipse3(outvid[f],ellipse,Tnum)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    
    
putTubesOnVid(paths, 'virat7_outstage1.mp4')

FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [5]:
#Fixes the paths so that each color corresponds to a path
newpaths=[[]]

for i in range(1,len(paths)):
    for e in paths[i]:
        RGB = e[3]
        makePathFlag = True
        
        for newpath in newpaths:
            if not newpath or not(newpath[0][3] == RGB):
                continue
            else:
                newpath.append(e)
                makePathFlag = False
                break
        
        if (makePathFlag):
            E = [e]
            newpaths.append(E)

            
#Gets rid of duplicates in each path (took forever smh)
K = deepcopy(newpaths)
L = deepcopy(newpaths)
M = deepcopy(newpaths)
newpaths = []

for i in range(0,len(K)):
    bigA = []
    for j in range(0,len(K[i])):
        thise = K[i][j]
        A = [K[i].index(e) for e in K[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [K[i][t] for t in list(set(bigA))]
    newpaths.append(out)

#for i in range(0,len(K)):
#    print("used to be...", len(K[i]))
#    print("now is...", len(newpaths[i]))


#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one
L = deepcopy(newpaths)
newpaths = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            toRemove.extend(A[:-1])

    newpaths.append([p for p in L[i] if p not in toRemove])
    

putTubesOnVid(newpaths, 'virat7_outstage2.mp4')


FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [6]:
L = deepcopy(newpaths)
temporalcutoff = 20
newpaths = [p for p in L if len(p)> temporalcutoff]

for path in newpaths:
    path.sort(key = lambda x: x[0])

putTubesOnVid(newpaths, 'virat7_outstage3.mp4')

FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [7]:
L = deepcopy(newpaths)
tubes = []

#Each slice should be [f, ((X,Y),(b,A),rot), [Vxavg,Vyavg], [R,G,B]]

for i in range(0,len(L)):
    slicestoAdd = []
    for f in range(L[i][0][0], L[i][-1][0]):
        #chunk = [L[i].index(e) for e in L[i] if abs(e[0]-f) < 15]
        
        thisone = [e for e in newpaths[1] if e[0]==f]
        if not thisone: #if we don't have a slice at this frame
            thischunk = []
            a = 2
            while len(thischunk) < 2:
                thischunk = [L[i].index(e) for e in L[i] if abs(e[0]-f)<=a]
                a=a+2
                
            chunkX = [L[i][ind][1][0][0] for ind in thischunk]
            X = sum(chunkX)/len(chunkX)
            chunkY = [L[i][ind][1][0][1] for ind in thischunk]
            Y = sum(chunkY)/len(chunkY)
            chunksmallax = [L[i][ind][1][1][0] for ind in thischunk]
            smallax=max(chunksmallax)
            chunkbigax = [L[i][ind][1][1][1] for ind in thischunk]
            bigax=max(chunkbigax)
            chunkphi = [L[i][ind][1][2] for ind in thischunk]
            phi=max(chunkphi)
            chunkf = [L[i][ind][0] for ind in thischunk]
            Vx = (chunkX[-1] - chunkX[0])/(max(chunkf)-min(chunkf))
            Vy = (chunkY[-1] - chunkY[0])/(max(chunkf)-min(chunkf))
        
            thisslice = [f, ((X,Y),(smallax,bigax),phi), [Vx,Vy], L[i][0][3]]
            slicestoAdd.append(thisslice)
            
    L[i].extend(slicestoAdd)
            
tubes = L


#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one
L = deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            toRemove.extend(A[:-1])

    tubes.append([p for p in L[i] if p not in toRemove])
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
    
putTubesOnVid(tubes, 'virat7_outstage4.mp4')
putTubesOnVid(tubes, 'virat7_outstage4nums.mp4', True)

FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [11]:
for t in tubes:
    print("Tube", tubes.index(t), "has length", len(t))
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])

Tube 0 has length 67
Tube 1 has length 374
Tube 2 has length 32
Tube 3 has length 430
Tube 4 has length 24
Tube 5 has length 32
Tube 6 has length 64
Tube 7 has length 164
Tube 8 has length 25
Tube 9 has length 142
Tube 10 has length 76
Tube 11 has length 193
Tube 12 has length 320
Tube 13 has length 482
Tube 14 has length 50
Tube 15 has length 30
Tube 16 has length 237
Tube 17 has length 109
Tube 18 has length 127


In [8]:
#Time to do some tube merging...

def getMerges(L, lookahead):
    mergeList = []
    for i in range(0,len(L)):
        CVx = [L[i][ind][2][0] for ind in range(-5,0)]
        CVy = [L[i][ind][2][1] for ind in range(-5,0)]
        forVx = sum(CVx)/len(CVx)
        forVy = sum(CVy)/len(CVy)  #ending velocity of tube i
        BVx = [L[i][ind][2][0] for ind in range(0,5)]
        BVy = [L[i][ind][2][1] for ind in range(0,5)]
        backVx = sum(BVx)/len(BVx)
        backVy = sum(BVy)/len(BVy)  #starting velocity of tube i

        forX = [L[i][ind][1][0][0] for ind in range(-10,0)]
        forY = [L[i][ind][1][0][1] for ind in range(-10,0)]
        backX = [L[i][ind][1][0][0] for ind in range(0,10)]
        backY = [L[i][ind][1][0][1] for ind in range(0,10)]
        
        forVx = (forX[-1] - forX[0]) /10
        forVy = (forY[-1] - forY[0]) /10
        backVx = (backX[-1] - backX[0]) /10
        backVy = (backY[-1] - backY[0]) /10

        predx = (sum(forX)/len(forX)) + (forVx*lookahead)
        predy = (sum(forY)/len(forY)) + (forVy*lookahead)
        predf = L[i][-1][0] + lookahead
        backpredx = (sum(backX)/len(backX)) - (backVx*lookahead)
        backpredy = (sum(backY)/len(backY)) - (backVy*lookahead)
        backpredf = L[i][0][0] - lookahead

        for j in range(0,len(tubes)):
            if i==j:
                continue

            closeones = [e for e in L[j] if abs(e[0] - predf)<5]
            Bcloseones= [e for e in L[j] if abs(e[0]-backpredf)<5]
            matchFlag = False
            BmatchFlag= False

            for e in closeones:
                r = math.sqrt((e[1][0][0] - predx)**2 + (e[1][0][1] - predy)**2 )
                if (r<100) and (abs(e[2][0] - forVx)<2) and (abs(e[2][1] - forVy)<2):
                    matchFlag = True

            for e in Bcloseones:
                r = math.sqrt((e[1][0][0] - backpredx)**2 + (e[1][0][1] - backpredy)**2 )
                if (r<50) and (abs(e[2][0] - backVx)<2) and (abs(e[2][1] - backVy)<2):
                    BmatchFlag = True

            if (matchFlag):
                mergeList.append([i,j,0])  #j goes in back of i
            if (BmatchFlag):
                mergeList.append([i,j,1])  #j goes in front of i

    nahList = []
    for m in mergeList:
        a = m[0]
        b = m[1]
        
        allVxa = [L[a][ind][2][0] for ind in range(0,len(L[a])) ] 
        Vxa = sum(allVxa)/len(allVxa)
        allVya = [L[a][ind][2][1] for ind in range(0,len(L[a])) ] 
        Vya = sum(allVya)/len(allVya)  

        allVxb = [L[b][ind][2][0] for ind in range(0,len(L[b])) ] 
        Vxb = sum(allVxb)/len(allVxb)
        allVyb = [L[b][ind][2][1] for ind in range(0,len(L[b])) ] 
        Vyb = sum(allVyb)/len(allVyb)

        if (Vxa/Vxb>0) and (Vya/Vyb>0 or (abs(Vya)<1.2 and abs(Vyb)<1.2) ) :
            A = [L[a][ind][0] for ind in range(0,len(L[a]))]
            B = [L[b][ind][0] for ind in range(0,len(L[b]))]
            overlap = [val for val in A if val in B]
            if len(overlap) > min(len(A),len(B))/2:
                nahList.append(m)
        else:
            nahList.append(m)

    mergeList = [m for m in mergeList if m not in nahList]
    
    return mergeList

def doMerge(L, mergeList):
    
    colorList = [t[0][3] for t in L]
    for m in mergeList:
        tlist = []
        for i in range(0,len(L)):
            for e in L[i]:
                if e[3]==colorList[m[0]] or e[3]==colorList[m[1]]:
                    tlist.append(i)

        flattubes = [ell for tube in L for ell in tube]
        cList = [colorList[a] for a in set(tlist)]
        connected = [e for e in flattubes if e[3] in cList]
        for i in set(tlist):
            L[i] = connected
    
    #Remove the extras (duplicates)
    delList = []
    for i in range(0,len(L)):
        if not(L.index(L[i]) == i):
            delList.append(i)
            
    print(delList)
    Otubes = [L[i] for i in range(0,len(L)) if i not in delList ]
    
    for t in Otubes:
        for e in t:
            e[3] = t[0][3]
    

    return Otubes


In [9]:
L = deepcopy(tubes)
lookahead = 5

M5 = getMerges(L,lookahead)
tubes = doMerge(L,M5)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)

[6, 7, 8, 10]


In [10]:
putTubesOnVid(tubes, 'virat7_outstage5.mp4')
putTubesOnVid(tubes, 'virat7_outstage5nums.mp4', True)

FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [11]:
L = deepcopy(tubes)
lookahead = 60

M60 = getMerges(L,lookahead)
newtubes = doMerge(L,M60)
print(M60)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(newtubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)
    
putTubesOnVid(tubes, 'virat7_outstage6.mp4')
putTubesOnVid(tubes, 'virat7_outstage6nums.mp4', True)

[13]
[[10, 13, 0]]
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062


In [12]:
for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)    
    
for t in tubes:
    if (len(t) >= (t[-1][0] - t[0][0] + 1)):
        continue
        
    else:
        print("Gaps in tube", tubes.index(t))
        toAdd = []
        have = [e[0] for e in t]
        donthave = [num for num in range(t[0][0], t[-1][0]) if num not in have]
        dlist = []
        for k, g in groupby(enumerate(donthave), lambda ix :ix[0]-ix[1]):
            dlist.append(list(map(itemgetter(1), g) ))
        for gap in dlist:
            preve = [e for e in t if e[0]==gap[0]-1][0]
            nexte = [e for e in t if e[0]==gap[-1]+1][0]
            fgap = len(gap)+1
            for f in gap:
                framediff = f - (gap[0]-1)
                
                X = (nexte[1][0][0] - preve[1][0][0]) * (framediff/fgap) + preve[1][0][0]
                Y = (nexte[1][0][1] - preve[1][0][1]) * (framediff/fgap) + preve[1][0][1]
                smallax = max(nexte[1][1][0],preve[1][1][0]) 
                bigax = max(nexte[1][1][1],preve[1][1][1]) 
                phi = (nexte[1][2] - preve[1][2]) * (framediff/fgap) + preve[1][2]
                Vx = (X - preve[1][0][0])/framediff
                Vy = (Y - preve[1][0][1])/framediff
                RGB = preve[3]
                
                newe = [f, ((X,Y), (smallax,bigax), phi), [Vx,Vy], RGB]
                toAdd.append(newe)
          
        t.extend(toAdd)  
            
for t in tubes:
    t.sort(key = lambda x: x[0])

    
putTubesOnVid(tubes, 'virat7_outstage7.mp4')
putTubesOnVid(tubes, 'virat7_outstage7nums.mp4', True)

Gaps in tube 1
Gaps in tube 2
Gaps in tube 3
Gaps in tube 4
Gaps in tube 10
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
FRAME 50 / 1062
FRAME 100 / 1062
FRAME 150 / 1062
FRAME 200 / 1062
FRAME 250 / 1062
FRAME 300 / 1062
FRAME 350 / 1062
FRAME 400 / 1062
FRAME 450 / 1062
FRAME 500 / 1062
FRAME 550 / 1062
FRAME 600 / 1062
FRAME 650 / 1062
FRAME 700 / 1062
FRAME 750 / 1062
FRAME 800 / 1062
FRAME 850 / 1062
FRAME 900 / 1062
FRAME 950 / 1062
FRAME 1000 / 1062
FRAME 1050 / 1062
